In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [17]:
def askisi5():

    class Counter:
        def __init__(self,count):
            self.count = 0
        
        def __str__(self):
            self.count+=1
            return "Hello World"+str(count)
    a=Counter()
    print(a)
    print(a)
    print(a)

askisi5()

NameError: name 'count' is not defined

#Below we see an implementation of the PLU factorization of a matrix , using only row exchange.

As known from Linear Algebra we can decompose a matrix $A$ with $PA=LU$ where $P$ is the product of permutation matrices , $L$ is a lower triangular matrix and $U$ an upper triangular matrix.

First , we calculate the max for every row in the column we are eliminating.Then we multiply the matrix $A$ with the permutation matrix needed in order to exchange rows.

After that we multiply the $L_{ij}^{-1}$ matrix (corresponding to each step's elimination matrix inverse) because we want to keep track of the permutations.At the final steps we want to have $P_nP_{n-1}..P_2P_1A=LU$.So in order to achieve that we want to do the following :
<br>
<br>
$$P_2\cdot E_{n1} \cdot E_{nn-1}... E_{31}\cdot E_{21}\cdot P_1\cdot A= P_2\cdot E_{32}\cdot E_{31}\cdot P_2^{-1}\cdot P_2\cdot P_1\cdot A$$
<br>
<br>
Because 
$E_{n1} \cdot E_{nn-1}... E_{31}\cdot E_{21}$ 
 are lower triangular matrices with all the non-zero values in the same column , the product $P_2\cdot E_{32}\cdot E_{31}\cdot P_2^{-1}$ , we also be an upper triangular matrix. And the total $L$ would be the inverse of all these products.

Total $L$ would have as values in every corresponding position $\frac{A_{ij}}{A_{pj}}$.

(We multiply the pivot row with $-\frac{A_{ij}}{A_{pj}}$ where $A_{ij}$ is the position we want to eliminate , and then add it to the row.)

Finally , we return the Permutation Matrix , the Lower Triangular and the Upper Triangular.

In [2]:
def PLU(A):
    steps=[]
    n=A.shape[0]
    U=np.zeros((n,n))
    L=np.eye(n,n)
    P_total=np.eye(n,n)
    A_original=A.copy()
    for i in range(n-1):
        #Finding the max element's index for the current column
        max_index=np.argmax(np.abs(A[i:,i]))

        #Defining a permutation matrix for the current step
        P=np.eye(n,n)
        P[i,:],P[max_index+i,:]=P[max_index+i,:].copy(),P[i,:].copy()

        #With every permutation we have to multiply L inverse of the previous step with
        #the new permutation , PLP^-1 , in order to finish with Pn,Pn-1,PA=LU
        A=np.dot(P,A)
        L=np.dot(P,L)
        L=np.dot(L,P.T)

        #We also keep track of the permutations in a P_total matrix

        P_total=np.dot(P,P_total)

        #We multiply the pivot element's row with A[i+1],i/A[i,i] , and substract it from the row
        #we want to eliminate. We know that the inverse of L_step would have the oposite signs of A[i+1],i/A[i,i]
        #So we store that into our Lower Triangular array.#
        for j in range(i+1,n):
            pivot=-(A[j,i]/A[i,i])*A[i,:]
            L[j,i]=((A[j,i]/A[i,i]))
            A[j,:]+=pivot
    return P_total,L,A


#Test problems for our method

In [3]:
A = np.array([[2, -1, -2], [-4, 6, 3], [-4, -2, 8]])
B=np.array([
    [2, -1, 0, 0],
    [-1, 2, -1, 0],
    [0, -1, 2, -1],
    [0, 0, -1, 2]
])

P,L,U=PLU(B)

print('The original matrix B\n'+str(B))
print('The product PLU after the decomposition \n'+str(np.dot(P.T,np.dot(L,U))))
print('The inverse of the permutations we performed \n',P.T)
print('The Lower Triangular Matrix \n'+str(L))
print('The Upper Triangular Matrix \n'+str(U))

P2,L2,U2=PLU(A)

for i in range(2):
    print("\n")

print('The original matrix A\n'+str(A))
print('The product PLU after the decomposition \n'+str(np.dot(P2.T,np.dot(L2,U2))))
print('The inverse of the permutations we performed \n',P2.T)
print('The Lower Triangular Matrix \n'+str(L2))
print('The Upper Triangular Matrix \n'+str(U2))

The original matrix B
[[ 2 -1  0  0]
 [-1  2 -1  0]
 [ 0 -1  2 -1]
 [ 0  0 -1  2]]
The product PLU after the decomposition 
[[ 2. -1.  0.  0.]
 [-1.  2. -1.  0.]
 [ 0. -1.  2. -1.]
 [ 0.  0. -1.  2.]]
The inverse of the permutations we performed 
 [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
The Lower Triangular Matrix 
[[ 1.          0.          0.          0.        ]
 [-0.5         1.          0.          0.        ]
 [ 0.         -0.66666667  1.          0.        ]
 [ 0.          0.         -0.75        1.        ]]
The Upper Triangular Matrix 
[[ 2.         -1.          0.          0.        ]
 [ 0.          1.5        -1.          0.        ]
 [ 0.          0.          1.33333333 -1.        ]
 [ 0.          0.          0.          1.25      ]]




The original matrix A
[[ 2 -1 -2]
 [-4  6  3]
 [-4 -2  8]]
The product PLU after the decomposition 
[[ 2. -1. -2.]
 [-4.  6.  3.]
 [-4. -2.  8.]]
The inverse of the permutations we performed 
 [[0. 0. 1.]
 [1. 0. 0.]
 [

#We can see that our method is also precise for n=1000 based on the results.

PLU decomposition requires $O(\frac{2}{3}n^{3})$ calculations in order to decompose a random matrix $A$.
So for big dimensions the method is quite slow.

In [4]:
def random_square_matrix(size):
    return np.random.rand(size, size)

random_matrix = random_square_matrix(1000)

P2,L2,U2=PLU(random_matrix)

for i in range(2):
    print("\n")

print('The original matrix A\n'+str(random_matrix))
print('The product PLU after the decomposition \n'+str(np.dot(P2.T,np.dot(L2,U2))))
print('The inverse of the permutations we performed \n',P2.T)
print('The Lower Triangular Matrix \n'+str(L2))
print('The Upper Triangular Matrix \n'+str(U2))






The original matrix A
[[0.26042719 0.66575804 0.22289115 ... 0.56249125 0.80059942 0.3076568 ]
 [0.24836438 0.39494713 0.50883941 ... 0.27549763 0.18607133 0.41975156]
 [0.95905598 0.89238934 0.04169042 ... 0.54649771 0.31686373 0.972523  ]
 ...
 [0.28310742 0.76370985 0.33605297 ... 0.74725802 0.35815977 0.83916819]
 [0.76234028 0.67676161 0.70024075 ... 0.13394612 0.74186287 0.16265763]
 [0.22267421 0.88018442 0.04005583 ... 0.2216299  0.63714692 0.70565362]]
The product PLU after the decomposition 
[[0.26042719 0.66575804 0.22289115 ... 0.56249125 0.80059942 0.3076568 ]
 [0.24836438 0.39494713 0.50883941 ... 0.27549763 0.18607133 0.41975156]
 [0.95905598 0.89238934 0.04169042 ... 0.54649771 0.31686373 0.972523  ]
 ...
 [0.28310742 0.76370985 0.33605297 ... 0.74725802 0.35815977 0.83916819]
 [0.76234028 0.67676161 0.70024075 ... 0.13394612 0.74186287 0.16265763]
 [0.22267421 0.88018442 0.04005583 ... 0.2216299  0.63714692 0.70565362]]
The inverse of the permutations we performed 